In [1]:
from utils import *

In [2]:
import lutorpy as lua
require("nn")
require("optim")
require("cutorch")
require("cunn")

True

In [3]:
np.random.seed(0)

In [14]:
location = np.load('src/pedestrians_location.npy')
paths = np.load('src/all_not_ext_paths_with_ids.npy')

In [5]:
M = M_star = 5
N = NUMBER_OF_PEDESTRIANS
X = FRAME_WIDTH
Y = FRAME_HEIGHT

In [6]:
disp_volume = np.zeros((X, Y, 2*M))

In [7]:
def get_t(curr_time, M, M_star):
    step = 20
    return np.arange(curr_time - step*(M-1), curr_time + step*(M_star+1), step)

In [8]:
t = get_t(100, M, M_star)
t

array([ 20,  40,  60,  80, 100, 120, 140, 160, 180, 200])

In [34]:
ind_t = (t/20).astype(int)

In [17]:
pedestrians_in_scene = paths[paths[:,1] == t[M-1]]

In [92]:
for p in pedestrians_in_scene:
    l = location[p[0],ind_t[:M]]
    # to fill zeros do something like that:
    # for i, row in enumerate(l):
    #     if (row == 0).all():
    #         l[i] = l[-1]
    d = (l[-1] - l) / np.array([X, Y]) + 1
    disp_volume[p[2], p[3]] = d.flatten()

In [146]:
xs, ys = disp_volume[:,:,0].nonzero()

for i, j in zip(xs,ys):
    np.array([i,j]) - (disp_volume[i,j].reshape(-1,2) - 1) * np.array([X, Y])